In [11]:
import pandas as pd
import os
from pathlib import Path
import tokenisation as tkn
import api_calls as api
import running_model as rnm
import json
import regex as re

In [12]:
WRITE_DIR = Path("./data/results/prompt_1/gpt-3.5-turbo/speeches_2/")

In [13]:
with open("./data/prompts/prompt_1.txt", "r") as file:
    prompt_1 = file.read().splitlines()

In [15]:
data = pd.read_parquet("./data/scraping/all_fed_speeches.parquet")

In [9]:
# title_words = ["policy", "economy", "outlook", "economic", "brief", "summary", "monetary", "future", "economy", "unemployment", "inflation", "update"]
title_words = ["policy", "outlook", "economic", "brief", "summary", "monetary", "future", "economy", "unemployment", "inflation", "update"]

In [14]:
count = 0
for i in range(data.shape[0]):
    if not any(ele in data.iloc[i, 2].lower() for ele in title_words):
        continue
    title = re.search(r'.*\/([^\/]+)\.[^\.]+$', data.iloc[i,3]).group(1)
    if title == "kroszner2007116a":
        continue
    if os.path.exists(f"./data/results/prompt_1/gpt-3.5-turbo/speeches_2/gpt-3.5-turbo_prompt_1_{title}.json"):
        print(f"done {title}, moving on...")
        continue
    print(data.iloc[i,0], data.iloc[i, 2])
    result = []
    sentences = tkn.get_important_text(tkn.basic_tokeniser(data.iloc[i, -1]))
    raw_sentiments = await api.get_multiple_api_calls("gpt-3.5-turbo", prompt_1, sentences)
    labels = [rnm._find_label(sentiment[1]) for sentiment in raw_sentiments]
    reasons = [sentiment[0] for sentiment in raw_sentiments]
    for sentence, label, reason in zip(sentences, labels, reasons):
        result.append({"sentence": sentence, "label": label, "reason" : reason})
    with open(f"./data/results/prompt_1/gpt-3.5-turbo/speeches_2/gpt-3.5-turbo_prompt_1_{title}.json", "w") as file:
        json.dump(result, file)


done brainard20201117a, moving on...
done clarida20201019a, moving on...
done clarida20201014a, moving on...
done powell20201006a, moving on...
done brainard20200901a, moving on...
done clarida20200831a, moving on...
done powell20200827a, moving on...
done bowman20200826a, moving on...
done bowman20200819a, moving on...
done brainard20200813a, moving on...
done brainard20200806a, moving on...
done brainard20200714a, moving on...
done clarida20200616a, moving on...
done clarida20200521a, moving on...
done powell20200513a, moving on...
done powell20200409a, moving on...
done clarida20200225a, moving on...
done clarida20200221a, moving on...
done brainard20200221a, moving on...
done quarles20200206a, moving on...
done bowman20200116a, moving on...
done clarida20200109a, moving on...
done waller20211217a, moving on...
done bowman20211129a, moving on...
done clarida20211119a, moving on...
done waller20211119a, moving on...
done clarida20211108a, moving on...
done quarles20211020a, moving on

In [39]:
df = data.set_index("date")

In [42]:
result = []
title = "powell20230321a"
sentences = tkn.get_important_text(tkn.basic_tokeniser(df.loc["2022-03-21", "text"]))
raw_sentiments = await api.get_multiple_api_calls("gpt-3.5-turbo", prompt_1, sentences)
labels = [rnm._find_label(sentiment[1]) for sentiment in raw_sentiments]
reasons = [sentiment[0] for sentiment in raw_sentiments]
for sentence, label, reason in zip(sentences, labels, reasons):
    result.append({"sentence": sentence, "label": label, "reason" : reason})
with open(f"./data/results/prompt_1/gpt-3.5-turbo/speeches/gpt-3.5-turbo_prompt_1_{title}.json", "w") as file:
    json.dump(result, file)

2023-10-17 11:25:42 [info     ] completed 36 / 1               index=36 response_status=200 total=1
2023-10-17 11:25:42 [info     ] completed 41 / 1               index=41 response_status=200 total=1
2023-10-17 11:25:42 [info     ] completed 16 / 1               index=16 response_status=200 total=1
2023-10-17 11:25:42 [info     ] completed 8 / 1                index=8 response_status=200 total=1
2023-10-17 11:25:42 [info     ] completed 45 / 1               index=45 response_status=200 total=1
2023-10-17 11:25:42 [info     ] completed 7 / 1                index=7 response_status=200 total=1
2023-10-17 11:25:43 [info     ] completed 27 / 1               index=27 response_status=200 total=1
2023-10-17 11:25:43 [info     ] completed 46 / 1               index=46 response_status=200 total=1
2023-10-17 11:25:43 [info     ] completed 9 / 1                index=9 response_status=200 total=1
2023-10-17 11:25:43 [info     ] completed 25 / 1               index=25 response_status=200 total=1
202

In [36]:
df = data.set_index("date")

In [48]:
pd.DataFrame({"sentence" : tkn.get_important_text(tkn.basic_tokeniser(df.loc["2022-03-21"]["text"]))}).to_excel("test.xlsx")

In [28]:
any(ele in data.iloc[51, 2] for ele in title_words)

False

In [31]:
count

294

In [9]:
data["title"].to_csv("quick.csv")

In [24]:
data.iloc[51]

date                                     2006-04-13 00:00:00
speaker                               Governor Mark W. Olson
title                             Update on the U.S. economy
link                   /newsevents/speech/olson20060413a.htm
text       Recent trends in the U.S. economy have been, o...
Name: 51, dtype: object

In [1]:
import FedTools as ft

In [3]:
x = ft.MonetaryPolicyCommittee().find_statements()

Constructing links between 1994 and 2023
Extracting the past 200 FOMC Statements.
Retrieving articles.
........................................................................................................................................................................................................

In [8]:
x["FOMC_Statements"][-1]

/var/folders/0g/0pyvp71x00d07s_b28p42ctc0000gn/T/ipykernel_1981/535763434.py:1: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  x["FOMC_Statements"][-1]


"The Federal Reserve, the central bank of the United States, provides            the nation with a safe, flexible, and stable monetary and financial            system.  Federal Open Market Committee  Monetary Policy Principles and Practice  Policy Implementation  Reports  Review of Monetary Policy Strategy, Tools, and Communications  Institution Supervision  Reports  Reporting Forms  Supervision & Regulation Letters  Banking Applications & Legal Developments  Regulatory Resources  Banking & Data Structure  Financial Stability Assessments  Financial Stability Coordination & Actions  Reports  Regulations & Statutes  Payment Policies  Reserve Bank Payment Services & Data  Financial Market Utilities & Infrastructures  Research, Committees, and Forums  Working Papers and Notes  Data, Models and Tools  Bank Assets and Liabilities  Bank Structure Data  Business Finance  Dealer Financing Terms  Exchange Rates and International Data  Financial Accounts  Household Finance  Industrial Activity  I

In [9]:
x

,FOMC_Statements
1994-02-04,Chairman Alan Greenspan announced today that t...
1994-03-22,Chairman Alan Greenspan announced today that t...
1994-04-18,The Federal Reserve today announced two action...
1994-05-17,Chairman Alan Greenspan announced today that t...
1994-08-16,The Federal Reserve announced today the follow...
...,...
2023-03-22,"The Federal Reserve, the central bank of the U..."
2023-05-03,"The Federal Reserve, the central bank of the U..."
2023-06-14,"The Federal Reserve, the central bank of the U..."
2023-07-26,"The Federal Reserve, the central bank of the U..."
